In [ ]:
# Importing Lib and data
import pandas as pd
from sklearn.cross_validation import train_test_split
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import math
from scipy import stats
from sklearn.utils.multiclass import type_of_target
import os

os.chdir("C:\\Mani\\Data_Science\\Final_project_Capstone")
Data=pd.read_csv("Data_Modeling_Resp1_v4.csv")

In [ ]:
Data.decribe()

In [ ]:
Data.shape

In [ ]:
# EDA 
Data['Response1'].value_counts()

In [ ]:
sns.countplot(x='Occupancy_Status',data=Data, palette='hls')
plt.show()
plt.savefig('count_plot')

In [ ]:
sns.countplot(x='season1',data=Data, palette='hls')
plt.show()
plt.savefig('count_plot')

In [ ]:
%matplotlib inline
pd.crosstab(Data.season1,Data.Response1).plot(kind='bar')
plt.title('Assets sold frequency')
plt.xlabel('season1')
plt.ylabel('Assets sold')
plt.savefig('Assets sold within and after 90 days')

In [ ]:
%matplotlib inline
pd.crosstab(Data.Rural_Urban,Data.Response1).plot(kind='bar')
plt.title('Assets sold frequency')
plt.xlabel('Rural_Urban')
plt.ylabel('Assets sold')
plt.savefig('Assets sold within and after 90 days')

In [ ]:
%matplotlib inline
pd.crosstab(Data.Bath_Count,Data.Response1).plot(kind='bar')
plt.title('Assets sold frequency')
plt.xlabel('Bath_Count')
plt.ylabel('Assets sold')
plt.savefig('Assets sold within and after 90 days')

In [ ]:
%matplotlib inline
pd.crosstab(Data.Bed_Count,Data.Response1).plot(kind='bar')
plt.title('Assets sold frequency')
plt.xlabel('Bed_Count')
plt.ylabel('Assets sold')
plt.savefig('Assets sold within and after 90 days')

In [ ]:
%matplotlib inline
pd.crosstab(Data.Occupancy_Statuswoe,Data.Response1).plot(kind='bar')
plt.title('Assets sold frequency')
plt.xlabel('Occupancy_Statuswoe')
plt.ylabel('Assets sold')
plt.savefig('Assets sold within and after 90 days')

In [ ]:
%matplotlib inline
pd.crosstab(Data.season1woe,Data.Response1).plot(kind='bar')
plt.title('Assets sold frequency')
plt.xlabel('season1woe')
plt.ylabel('Assets sold')
plt.savefig('Assets sold within and after 90 days')

In [ ]:
# Calculating WOE and IV
class WOE:
    def __init__(self):
        self._WOE_MIN = -20
        self._WOE_MAX = 20

    def woe(self, X, y, event=1):
       
        self.check_target_binary(y)
        X1 = self.feature_discretion(X)

        res_woe = []
        res_iv = []
        for i in range(0, X1.shape[-1]):
            x = X1[:, i]
            woe_dict, iv1 = self.woe_single_x(x, y, event)
            res_woe.append(woe_dict)
            res_iv.append(iv1)
        return np.array(res_woe), np.array(res_iv)

    def woe_single_x(self, x, y, event=1):
       
        self.check_target_binary(y)

        event_total, non_event_total = self.count_binary(y, event=event)
        x_labels = np.unique(x)
        woe_dict = {}
        iv = 0
        for x1 in x_labels:
            y1 = y[np.where(x == x1)[0]]
            event_count, non_event_count = self.count_binary(y1, event=event)
            rate_event = 1.0 * event_count / event_total
            rate_non_event = 1.0 * non_event_count / non_event_total
            if rate_event == 0:
                woe1 = self._WOE_MIN
            elif rate_non_event == 0:
                woe1 = self._WOE_MAX
            else:
                woe1 = math.log(rate_event / rate_non_event)
            woe_dict[x1] = woe1
            iv += (rate_event - rate_non_event) * woe1
        return woe_dict, iv

    def woe_replace(self, X, woe_arr):
        
      
        if X.shape[-1] != woe_arr.shape[-1]:
            raise ValueError('WOE dict array')

        res = np.copy(X).astype(float)
        idx = 0
        for woe_dict in woe_arr:
            for k in woe_dict.keys():
                woe = woe_dict[k]
                res[:, idx][np.where(res[:, idx] == k)[0]] = woe * 1.0
            idx += 1

        return res

    def combined_iv(self, X, y, masks, event=1):
        
        if masks.shape[-1] != X.shape[-1]:
            raise ValueError('Masks array length')

        x = X[:, np.where(masks == 1)[0]]
        tmp = []
        for i in range(x.shape[0]):
            tmp.append(self.combine(x[i, :]))

        dumy = np.array(tmp)
        # dumy_labels = np.unique(dumy)
        woe, iv = self.woe_single_x(dumy, y, event)
        return woe, iv

    def combine(self, list):
        res = ''
        for item in list:
            res += str(item)
        return res

    def count_binary(self, a, event=1):
        event_count = (a == event).sum()
        non_event_count = a.shape[-1] - event_count
        return event_count, non_event_count

    def check_target_binary(self, y):
        
        y_type = type_of_target(y)
        if y_type not in ['binary']:
            raise ValueError('Label type must be binary')

    def feature_discretion(self, X):
        
        temp = []
        for i in range(0, X.shape[-1]):
            x = X[:, i]
            x_type = type_of_target(x)
            if x_type == 'continuous':
                x1 = self.discrete(x)
                temp.append(x1)
            else:
                temp.append(x)
        return np.array(temp).T

    def discrete(self, x):
        
        res = np.array([0] * x.shape[-1], dtype=int)
        for i in range(5):
            point1 = stats.scoreatpercentile(x, i * 20)
            point2 = stats.scoreatpercentile(x, (i + 1) * 20)
            x1 = x[np.where((x >= point1) & (x <= point2))]
            mask = np.in1d(x, x1)
            res[mask] = (i + 1)
        return res

    @property
    def WOE_MIN(self):
        return self._WOE_MIN
    @WOE_MIN.setter
    def WOE_MIN(self, woe_min):
        self._WOE_MIN = woe_min
    @property
    def WOE_MAX(self):
        return self._WOE_MAX
    @WOE_MAX.setter
    def WOE_MAX(self, woe_max):
        self._WOE_MAX = woe_max

In [ ]:
# Test and Tain data spilt
train, test = train_test_split(Data, train_size=0.80, random_state=1)

In [ ]:
qualityTrain = pd.DataFrame(train, columns=Data.columns)
qualityTest = pd.DataFrame(test, columns=Data.columns)

In [ ]:
cols = ['Agent_Prop_Conditionwoe','state1woe','PROPERTY_TYPEwoe','Census_Urban_Ruralwoe',
 'Occupancy_Statuswoe',
 'season1woe',
 'Rural_Urbanwoe',
 'bath_binwoe',
 'bed_binwoe',
 'age_yr_bin1woe',
 'upb_fc_bin1woe',
 'sqft_drift_binwoe',
 'price_drift_binwoe',
'state_drift_bin1woe']

In [ ]:
x = qualityTest[cols]
x = sm.add_constant(x)
y = qualityTest['Response1']
z = qualityTest['Response1']

In [ ]:
model1 = sm.Logit(y, x.astype(float)).fit()

In [ ]:
print(model1.summary())

In [ ]:
pred = model1.predict(x)
pred=np.where(pred > 0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(z, pred)
cm

In [ ]:
# Precision(sensitivity), Recall(specificity)
from sklearn.metrics import classification_report
print(classification_report(z, pred))

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(z, pred)
roc_auc=metrics.auc(fpr, tpr)
roc_auc

In [ ]:
# Confusion Matrix
import matplotlib.pyplot as plt
plt.title('Sold and Unsold assest characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 2], [0, 2],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('Logistic_ROC_curve.pdf') 
plt.show()
plt.close()

In [ ]:
# Accuracy 
from sklearn.metrics import accuracy_score
print (accuracy_score(z, pred))

In [ ]:
GINI = (2 * roc_auc) - 1
print(GINI)
